<a href="https://colab.research.google.com/github/Nastenkaslastenka/NS/blob/main/%D0%9F%D0%BB%D1%83%D0%B6%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2%D0%B0_%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import numpy as np
!pip install torchmetrics
!pip install optuna
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 25.9 MB/s eta 0:00:00


In [28]:
import torchmetrics
import optuna

In [29]:
# Настроим устройство для вычислений
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

DATA_ROOT = "/content/gdrive/MyDrive/EYES/"

Mounted at /content/gdrive/


In [31]:
# Заданные параметры
BATCH_SIZE = 32
N_EPOCHS = 30
TEST_TARGET_ACCURACY = 93.0  # Минимальная необходимая точность на тесте

In [32]:
# Преобразования изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Масштабируем изображения
    transforms.ToTensor(),           # Преобразуем в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализуем цвета
])


In [33]:
# Подготавливаем данные
full_dataset = datasets.ImageFolder(root=DATA_ROOT, transform=transform)

In [34]:
# Разделяем на train, valid и test
train_idx, temp_idx = train_test_split(list(range(len(full_dataset))), test_size=0.2, random_state=42)
valid_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

train_set = torch.utils.data.Subset(full_dataset, train_idx)
valid_set = torch.utils.data.Subset(full_dataset, valid_idx)
test_set = torch.utils.data.Subset(full_dataset, test_idx)

In [35]:
# Создаем loaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)

In [36]:
# Архитектура модели
class DiabeticRetinopathyModel(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super(DiabeticRetinopathyModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_layer_stack(x)

In [37]:
# Вспомогательные функции
def train_step(model, dataloader, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            loss_value = loss.item()
            current = batch * len(X)
            print(f"Loss: {loss_value:.4f} | Progress: {current}/{size}")

In [38]:
def eval_step(model, dataloader, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return correct, test_loss

In [39]:
# Основная функция обучения
def train_and_evaluate(model, train_dl, valid_dl, epochs, loss_fn, optimizer, device):
    results = {'train_loss': [], 'train_acc': [], 'valid_loss': [], 'valid_acc': []}
    best_acc = float('-inf')
    best_weights = None
    for t in range(epochs):
        print(f"Epoch {t+1}\n----------------------------")
        train_step(model, train_dl, loss_fn, optimizer, device)
        train_acc, train_loss = eval_step(model, train_dl, loss_fn, device)
        valid_acc, valid_loss = eval_step(model, valid_dl, loss_fn, device)
        print(f"Train Acc: {(100*train_acc):>0.1f}%, Valid Acc: {(100*valid_acc):>0.1f}%\n")
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['valid_loss'].append(valid_loss)
        results['valid_acc'].append(valid_acc)
        if valid_acc > best_acc:
            best_acc = valid_acc
            best_weights = model.state_dict()
    return results, best_weights

In [40]:
# Функция поиска гиперпараметров
def objective(trial):
    # Предлагаемые гиперпараметры
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    units = trial.suggest_int('units', 64, 512)

    # Создание модели
    model = DiabeticRetinopathyModel(input_shape=224*224*3, hidden_units=units, output_shape=5).to(DEVICE)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Обучение и проверка
    results, _ = train_and_evaluate(model, train_loader, valid_loader, N_EPOCHS, loss_fn, optimizer, DEVICE)
    valid_acc = results['valid_acc'][-1]
    return valid_acc


In [43]:
# Поиск наилучших гиперпараметров
N_EPOCHS = 5
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)

[I 2025-10-23 22:18:19,109] A new study created in memory with name: no-name-cfd00abb-432e-4bc0-a313-86f4c94d2df0
/tmp/ipython-input-1044294083.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


Epoch 1
----------------------------
Loss: 1.8601 | Progress: 0/5617
Loss: 86.2546 | Progress: 320/5617
Loss: 454.0627 | Progress: 640/5617
Loss: 114.1312 | Progress: 960/5617
Loss: 45.3113 | Progress: 1280/5617
Loss: 89.2683 | Progress: 1600/5617
Loss: 3.7051 | Progress: 1920/5617
Loss: 3.0538 | Progress: 2240/5617
Loss: 4.5475 | Progress: 2560/5617
Loss: 1.1593 | Progress: 2880/5617
Loss: 6.6497 | Progress: 3200/5617
Loss: 7.8568 | Progress: 3520/5617
Loss: 1.7017 | Progress: 3840/5617
Loss: 6.0292 | Progress: 4160/5617
Loss: 7.5037 | Progress: 4480/5617
Loss: 1.9341 | Progress: 4800/5617
Loss: 0.3692 | Progress: 5120/5617
Loss: 0.7157 | Progress: 5440/5617
Train Acc: 80.8%, Valid Acc: 76.9%

Epoch 2
----------------------------
Loss: 1.1633 | Progress: 0/5617
Loss: 0.5005 | Progress: 320/5617
Loss: 1.0085 | Progress: 640/5617
Loss: 0.5734 | Progress: 960/5617
Loss: 0.4206 | Progress: 1280/5617
Loss: 3.4334 | Progress: 1600/5617
Loss: 0.5318 | Progress: 1920/5617
Loss: 0.3165 | Progr

[I 2025-10-23 22:43:48,211] Trial 0 finished with value: 0.7863247863247863 and parameters: {'lr': 0.005555680078023373, 'units': 380}. Best is trial 0 with value: 0.7863247863247863.


Train Acc: 81.6%, Valid Acc: 78.6%



In [44]:
# Лучшие параметры
best_hyperparams = study.best_params
print(f"Best hyperparameters found:\nLearning Rate: {best_hyperparams['lr']},\nHidden Units: {best_hyperparams['units']}")

Best hyperparameters found:
Learning Rate: 0.005555680078023373,
Hidden Units: 380


In [45]:
# Обучение модели с лучшими параметрами
final_model = DiabeticRetinopathyModel(input_shape=224*224*3, hidden_units=best_hyperparams['units'], output_shape=5).to(DEVICE)
final_optimizer = torch.optim.Adam(final_model.parameters(), lr=best_hyperparams['lr'])
final_results, _ = train_and_evaluate(final_model, train_loader, valid_loader, N_EPOCHS, nn.CrossEntropyLoss(), final_optimizer, DEVICE)

Epoch 1
----------------------------
Loss: 1.7840 | Progress: 0/5617
Loss: 3703.1013 | Progress: 320/5617
Loss: 602.2808 | Progress: 640/5617
Loss: 118.1893 | Progress: 960/5617
Loss: 33.7468 | Progress: 1280/5617
Loss: 18.0976 | Progress: 1600/5617
Loss: 31.6233 | Progress: 1920/5617
Loss: 1.0080 | Progress: 2240/5617
Loss: 0.5584 | Progress: 2560/5617
Loss: 0.5347 | Progress: 2880/5617
Loss: 0.7297 | Progress: 3200/5617
Loss: 0.3733 | Progress: 3520/5617
Loss: 0.7262 | Progress: 3840/5617
Loss: 0.5339 | Progress: 4160/5617
Loss: 0.4458 | Progress: 4480/5617
Loss: 0.4759 | Progress: 4800/5617
Loss: 0.6451 | Progress: 5120/5617
Loss: 8.9009 | Progress: 5440/5617
Train Acc: 81.5%, Valid Acc: 78.6%

Epoch 2
----------------------------
Loss: 0.4561 | Progress: 0/5617
Loss: 0.3789 | Progress: 320/5617
Loss: 0.3921 | Progress: 640/5617
Loss: 0.5579 | Progress: 960/5617
Loss: 0.4713 | Progress: 1280/5617
Loss: 0.5680 | Progress: 1600/5617
Loss: 0.3899 | Progress: 1920/5617
Loss: 0.6433 | Pr

In [49]:
def detailed_eval_step(model, dataloader, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for i, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Отдельный вывод на каждой итерации
            acc = correct / ((i+1)*BATCH_SIZE)
            print(f"Batch {i+1}/{num_batches}: Correct: {correct:.0f}, Total: {min((i+1)*BATCH_SIZE, size)}, Accuracy: {acc*100:.2f}%")

    test_loss /= num_batches
    correct /= size
    return correct, test_loss

In [51]:
# Тестирование модели
test_acc, test_loss = detailed_eval_step(final_model, test_loader, nn.CrossEntropyLoss(), DEVICE)
print(f"Final Test Accuracy: {test_acc*100:.2f}%")
print(f"Final Test Loss: {test_loss:.4f}")

Batch 1/22: Correct: 28, Total: 32, Accuracy: 87.50%
Batch 2/22: Correct: 57, Total: 64, Accuracy: 89.06%
Batch 3/22: Correct: 83, Total: 96, Accuracy: 86.46%
Batch 4/22: Correct: 107, Total: 128, Accuracy: 83.59%
Batch 5/22: Correct: 130, Total: 160, Accuracy: 81.25%
Batch 6/22: Correct: 151, Total: 192, Accuracy: 78.65%
Batch 7/22: Correct: 179, Total: 224, Accuracy: 79.91%
Batch 8/22: Correct: 205, Total: 256, Accuracy: 80.08%
Batch 9/22: Correct: 228, Total: 288, Accuracy: 79.17%
Batch 10/22: Correct: 256, Total: 320, Accuracy: 80.00%
Batch 11/22: Correct: 280, Total: 352, Accuracy: 79.55%
Batch 12/22: Correct: 308, Total: 384, Accuracy: 80.21%
Batch 13/22: Correct: 334, Total: 416, Accuracy: 80.29%
Batch 14/22: Correct: 362, Total: 448, Accuracy: 80.80%
Batch 15/22: Correct: 390, Total: 480, Accuracy: 81.25%
Batch 16/22: Correct: 413, Total: 512, Accuracy: 80.66%
Batch 17/22: Correct: 437, Total: 544, Accuracy: 80.33%
Batch 18/22: Correct: 466, Total: 576, Accuracy: 80.90%
Batch 1

In [52]:
# Проверка достижения цели
if test_acc >= TEST_TARGET_ACCURACY / 100:
    print("Цель достигнута!")
else:
    print("Цель не достигнута.")

Цель не достигнута.


In [53]:
import copy
from torchvision import transforms
from torch.utils.data import SubsetRandomSampler

In [55]:
# Преобразование для теста с небольшим уровнем случайных модификаций
test_augment_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Немного меняем яркость и контраст
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [56]:
# Массив для хранения результатов по различным итерациям
accuracies = []
losses = []

In [58]:
# Количество попыток (увеличим длительность)
ITERATIONS = 5  # Можете задать большее значение, чтобы замедлить выполнение

for iter_num in range(ITERATIONS):
    # Создаём временный тестовый датасет
    temp_test_dataset = copy.deepcopy(test_set)
    temp_test_dataset.transform = test_augment_transforms

    # Меняем рандомную последовательность
    indices = list(range(len(temp_test_dataset)))
    np.random.shuffle(indices)
    sampler = SubsetRandomSampler(indices)
    temp_test_loader = DataLoader(temp_test_dataset, batch_size=32, sampler=sampler)

    # Проверяем модель на новом наборе данных
    test_acc, test_loss = detailed_eval_step(final_model, temp_test_loader, nn.CrossEntropyLoss(), DEVICE)
    accuracies.append(test_acc)
    losses.append(test_loss)
    print(f"Iteration {iter_num+1}: Final Test Accuracy: {test_acc*100:.2f}%, Loss: {test_loss:.4f}")

Batch 1/22: Correct: 26, Total: 32, Accuracy: 81.25%
Batch 2/22: Correct: 52, Total: 64, Accuracy: 81.25%
Batch 3/22: Correct: 79, Total: 96, Accuracy: 82.29%
Batch 4/22: Correct: 106, Total: 128, Accuracy: 82.81%
Batch 5/22: Correct: 131, Total: 160, Accuracy: 81.88%
Batch 6/22: Correct: 159, Total: 192, Accuracy: 82.81%
Batch 7/22: Correct: 186, Total: 224, Accuracy: 83.04%
Batch 8/22: Correct: 211, Total: 256, Accuracy: 82.42%
Batch 9/22: Correct: 234, Total: 288, Accuracy: 81.25%
Batch 10/22: Correct: 260, Total: 320, Accuracy: 81.25%
Batch 11/22: Correct: 285, Total: 352, Accuracy: 80.97%
Batch 12/22: Correct: 316, Total: 384, Accuracy: 82.29%
Batch 13/22: Correct: 340, Total: 416, Accuracy: 81.73%
Batch 14/22: Correct: 367, Total: 448, Accuracy: 81.92%
Batch 15/22: Correct: 392, Total: 480, Accuracy: 81.67%
Batch 16/22: Correct: 422, Total: 512, Accuracy: 82.42%
Batch 17/22: Correct: 451, Total: 544, Accuracy: 82.90%
Batch 18/22: Correct: 478, Total: 576, Accuracy: 82.99%
Batch 1

In [60]:
# Берём среднее значение по итогам всех итераций
mean_accuracy = np.mean(accuracies)
mean_loss = np.mean(losses)

print(f"Average Test Accuracy: {mean_accuracy*100:.2f}%")
print(f"Average Test Loss: {mean_loss:.4f}")

Average Test Accuracy: 81.79%
Average Test Loss: 1.3620
